Basándonos en la flota de Aerolíneas Argentinas, definimos los siguientes modelos de avión a modelar:

*   Embraer: capacidad de 96 pasajeros
*   Boeing 737: capacidad de 170 pasajeros
*   Airbus: capacidad de 220 pasajeros

En cada caso, estimamos que la ocupación con una lognormal centrada en la media 5% para representar la ocupación estimada promedio. Se truncan números mayores a la capacidad máxima.


In [ ]:
# -*- coding: utf-8 -*-
import time, json, datetime as dt
from typing import Tuple, List, Optional, Dict
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.animation as animation
from IPython.display import HTML
from collections import Counter
import random

In [ ]:
class Avion:
    """Representa un único avión en la simulación."""
    def __init__(self, id_avion, minuto_actual, pasajeros):
        self.id = id_avion
        self.distancia_a_aep = 100.0
        self.velocidad_actual = 0.0
        self.estado = "APROXIMANDO"
        self.minuto_creacion = minuto_actual
        self.pasajeros = pasajeros

    def __repr__(self):
        """Formato para imprimir el estado del avión de forma legible."""
        return (f"Avion(ID:{self.id}, Dist:{self.distancia_a_aep:.1f}mn, "
                f"Vel:{self.velocidad_actual:.0f}kt, Est:{self.estado})")

In [ ]:
MODELOS = {
    "Embraer": 96,
    "Boeing 737": 170,
    "Airbus": 220
}

VENTANILLAS_MIGRACIONES = 10
TASA_MIGRACIONES_MANUAL = VENTANILLAS_MIGRACIONES * 3  # personas por minuto

In [ ]:
def calcular_velocidad_maxima_permitida(distancia):
    """Devuelve la velocidad máxima según la distancia a AEP, en kt."""
    if distancia > 50: return 300
    elif distancia > 15: return 250
    elif distancia > 5: return 200
    else: return 150

def calcular_velocidad_minima_permitida(distancia):
    """Devuelve la velocidad mínima según la distancia a AEP, en kt."""
    if distancia > 50: return 250
    elif distancia > 15: return 200
    elif distancia > 5: return 150
    else: return 120

def calcular_separacion_en_tiempo(avion_atras, avion_adelante):
    """Calcula la separación en minutos entre dos aviones."""
    distancia_separacion = avion_atras.distancia_a_aep - avion_adelante.distancia_a_aep
    velocidad_referencia = avion_atras.velocidad_actual
    if velocidad_referencia == 0:
        return float('inf')
    return (distancia_separacion / velocidad_referencia) * 60

def distancia_aeroparque_free_flow(posicion, margen_minutos) -> float:
    """ Calcula cuánta distancia se recorre en margen_minutos desde posicion hacia AEP."""
    distancia_recorrida = calcular_velocidad_maxima_permitida(posicion) / 60 * margen_minutos
    return distancia_recorrida


def buscar_gap(avion, fila_aviones) -> bool:
    """ Indica si el avión puede reinsertarse con un gap de 10 minutos o no. """
    distancia_actual = avion.distancia_a_aep
    # Calculo cuánta distancia recorro desde mi posición actual yendo para adelante o atrás 5 min,
    # asumiento que se mantiene la velocidad máxima del tramo actual
    distancia_recorrida = distancia_aeroparque_free_flow(distancia_actual, margen_minutos=5)

    # Revisar que de posicion hacia +- distancia recorrida no haya aviones APROXIMANDO
    for otro_avion in fila_aviones:
      if otro_avion.estado == "APROXIMANDO" or otro_avion.estado == "AJUSTANDO_VELOCIDAD":
        # Tiene que estar entre: distancia_actual + distancia_recorrida >= otro_avion.distancia_a_aep >= distancia_actual - distancia_recorrida
        if distancia_actual + distancia_recorrida >= otro_avion.distancia_a_aep >= distancia_actual - distancia_recorrida:
          return False
    return True

In [ ]:
def gestionar_logica_aproximacion(avion, avion_de_adelante):
    """Toma todas las decisiones para un avión que está APROXIMANDO."""
    if avion_de_adelante is None:
        # Caso 1: Camino libre. Va a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
    else:
        tiempo_sep = calcular_separacion_en_tiempo(avion, avion_de_adelante)

        if tiempo_sep < 4:
            # Caso 2: Muy cerca. Intenta reducir velocidad.

            velocidad_req = avion_de_adelante.velocidad_actual - 20

            if velocidad_req < calcular_velocidad_minima_permitida(avion.distancia_a_aep):
                # Caso 3: No puede reducir lo suficiente. Se da la vuelta.
                avion.estado = "REGRESANDO"
                avion.velocidad_actual = 200
                print(f" (!) Avion {avion.id} inicia maniobra de regreso.")
            else:
                # Puede reducir de forma segura.
                avion.estado = "AJUSTANDO_VELOCIDAD"
                avion.velocidad_actual = velocidad_req
        else:
            # Hay espacio suficiente. Va a máxima velocidad.
            avion.estado = "APROXIMANDO"
            avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)

def gestionar_logica_regreso(avion_regresando, todos_los_aviones, resultados=None, minuto=None, warmup=0):
    """
    Gestiona un avión REGRESANDO. Primero busca un hueco para reingresar.
    Si no lo encuentra, chequea si debe desviarse.
    """
    # 1. Si el bucle termina sin encontrar hueco, se chequea el desvío
    if avion_regresando.distancia_a_aep > 100:
        avion_regresando.estado = "DESVIADO"
        print(f" (X) Avion {avion_regresando.id} no encontró hueco y se desvió a Montevideo.")

    # 2. Buscar un hueco de 10 minutos entre cada par de aviones
    elif buscar_gap(avion_regresando, todos_los_aviones):
        # 3. Si se encuentra un hueco, el avión reingresa
        avion_regresando.estado = "APROXIMANDO"
        print(f" (O) ¡Avion {avion_regresando.id} encontró un hueco y reingresa a la fila!")

In [ ]:
def generar_nuevos_aviones(minuto, aviones, next_id, probabilidad):
    """Decide si un nuevo avión aparece en el horizonte."""
    if np.random.uniform(0,1) < probabilidad:
        # Generamos el modelo de avión con su cantidad de pasajeros
        modelo = np.random.choice(list(MODELOS.keys()))
        capacidad = MODELOS[modelo]

        # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
        pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

        nuevo_avion = Avion(next_id, minuto, pasajeros_a_bordo)
        aviones.append(nuevo_avion)
        print(f"Min {minuto}: -> Aparece Avion {next_id} con {pasajeros_a_bordo} pasajeros en el horizonte.")
        return next_id + 1
    return next_id

def actualizar_estados_y_velocidades(aviones, resultados=None, minuto=None, warmup=0):
    """Actualiza el estado y la velocidad de cada avión según las reglas."""
    aviones.sort(key=lambda avion: avion.distancia_a_aep)
    for i, avion in enumerate(aviones):
        if avion.estado == "REGRESANDO":
            gestionar_logica_regreso(avion, aviones, resultados, minuto, warmup)

        else: # APROXIMANDO o AJUSTANDO_VELOCIDAD
            avion_de_adelante = aviones[i-1] if i > 0 and aviones[i-1].estado != "REGRESANDO" else None
            gestionar_logica_aproximacion(avion, avion_de_adelante)

        if resultados is not None and avion.estado == "AJUSTANDO_VELOCIDAD" and minuto is not None and minuto >= warmup:
            resultados['congestion_events'] += 1

def mover_aviones(aviones):
    """Mueve cada avión según su velocidad final para este minuto."""
    for avion in aviones:
        distancia_recorrida = avion.velocidad_actual / 60
        if avion.estado == "REGRESANDO":
            avion.distancia_a_aep += distancia_recorrida
        else:
            avion.distancia_a_aep -= distancia_recorrida

def gestionar_aviones_finalizados(minuto, aviones, resultados=None, tiempo_ideal=23.4, warmup=0):
    aviones_activos = []
    aviones_finalizados_hoy = []
    for avion in aviones:
        if avion.distancia_a_aep <= 0:
            avion.estado = "ATERRIZADO"
            # Solo calculamos y contamos si nos pasaron el diccionario de resultados
            if resultados is not None and minuto is not None and minuto >= warmup:
                tiempo_real = minuto - avion.minuto_creacion
                retraso = max(0, tiempo_real - tiempo_ideal)
                resultados['total_delay_min'] += retraso
            aviones_finalizados_hoy.append(avion)
        elif avion.estado == "DESVIADO":
            aviones_finalizados_hoy.append(avion)
        else:
            aviones_activos.append(avion)
    return aviones_activos, aviones_finalizados_hoy

def guardar_datos_del_minuto(minuto, aviones, historial):
    """Guarda el estado de cada avión en el minuto actual."""
    for avion in aviones:
        historial.append({
            'minuto': minuto,
            'id': avion.id,
            'distancia': avion.distancia_a_aep,
            'estado': avion.estado
        })

In [ ]:
def simular_una_corrida(lambda_val, tiempo_total=1080, warmup=0):
    """
    Ejecuta una corrida completa de la simulación para un lambda dado
    y devuelve un diccionario con los resultados.
    """
    # --- Parámetros y Almacenamiento ---
    aviones_activos = []
    aviones_finalizados = []
    cola_migraciones = 0
    next_avion_id = 1
    TIEMPO_IDEAL_VIAJE = 23.4  # Nuestro baseline en minutos

    # --- Contadores para ESTA corrida específica ---
    resultados_de_la_corrida = {
        'total_delay_min': 0.0,
        'congestion_events': 0,
        'diversions': 0,
        'landed_planes': 0,
        'inbound_planes': 0,
        'total_planes': 0,
        'active_planes':0,
        'returning_planes':0,
        'queue_size':0
    }

    historial_metricas = {
        'minuto': [],
        'delay_acum': [],
        'congestion_events': [],
        'diversions': [],
        'landed_planes': [],
        'delay_avg': [],
        'inbound_planes': [],
        'active_planes':[],
        'returning_planes':[],
        'queue_size':[]
    }

    # --- Bucle de un día de simulación ---
    for minuto in range(tiempo_total):
        # Generar arribos (ahora pasa el minuto de creación)
        if random.random() < lambda_val:
            # Generamos el modelo de avión con su cantidad de pasajeros
            modelo = np.random.choice(list(MODELOS.keys()))
            capacidad = MODELOS[modelo]

            # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
            pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

            nuevo_avion = Avion(next_avion_id, minuto, pasajeros_a_bordo)
            aviones_activos.append(nuevo_avion)
            next_avion_id += 1
            if minuto >= warmup:
                resultados_de_la_corrida['total_planes'] += 1


        # Actualizar estados y contar congestiones
        actualizar_estados_y_velocidades(aviones_activos, resultados_de_la_corrida, minuto=minuto, warmup=warmup)

        # Mover aviones
        mover_aviones(aviones_activos)

        # Gestionar aviones que terminaron (y contar retrasos, desvíos, aterrizajes)
        activos_actualizados, finalizados_ahora = gestionar_aviones_finalizados(
            minuto, aviones_activos, resultados_de_la_corrida, TIEMPO_IDEAL_VIAJE, warmup=warmup
        )
        aviones_activos = activos_actualizados
        aviones_finalizados.extend(finalizados_ahora)

        # Bajar pasajeros a la cola de migraciones
        for avion in finalizados_ahora:
            if avion.estado == "ATERRIZADO":
                cola_migraciones += avion.pasajeros
        
        # Actualizar cola de migraciones a tasa de personas por minuto
        cola_migraciones = max(0, cola_migraciones - TASA_MIGRACIONES_MANUAL)

        if minuto >= warmup:
            resultados_de_la_corrida['queue_size'] = cola_migraciones

        # Guardar métricas del minuto (solo si pasó el warm-up)
        if minuto >= warmup:
            for avion in aviones_activos:
                if avion.estado == "APROXIMANDO" or avion.estado == "AJUSTANDO_VELOCIDAD":
                    resultados_de_la_corrida['inbound_planes'] += 1
                elif avion.estado == "REGRESANDO":
                    resultados_de_la_corrida['returning_planes'] += 1

            for avion in finalizados_ahora:
                if avion.estado == "DESVIADO":
                    resultados_de_la_corrida['diversions'] += 1
                elif avion.estado == "ATERRIZADO":
                    resultados_de_la_corrida['landed_planes'] += 1

        historial_metricas['minuto'].append(minuto)
        historial_metricas['delay_acum'].append(resultados_de_la_corrida['total_delay_min'])
        historial_metricas['congestion_events'].append(resultados_de_la_corrida['congestion_events'])
        historial_metricas['diversions'].append(resultados_de_la_corrida['diversions'])
        historial_metricas['landed_planes'].append(resultados_de_la_corrida['landed_planes'])
        landed = resultados_de_la_corrida['landed_planes']
        if landed > 0:
            avg_delay = resultados_de_la_corrida['total_delay_min'] / landed
        else:
            avg_delay = 0.0
        historial_metricas['delay_avg'].append(avg_delay)
        historial_metricas['inbound_planes'].append(resultados_de_la_corrida['inbound_planes'])
        historial_metricas['active_planes'].append(len(aviones_activos))
        historial_metricas['returning_planes'].append(resultados_de_la_corrida['returning_planes'])
        historial_metricas['queue_size'].append(resultados_de_la_corrida['queue_size'])

    resultados_de_la_corrida['active_planes'] = sum(historial_metricas['active_planes'])

    return resultados_de_la_corrida, pd.DataFrame(historial_metricas)

def ejecutar_experimentos():
    """
    Función principal que ejecuta el bucle experimental para varios lambdas.
    """
    # --- Configuración del Experimento ---
    lambdas_a_probar = [0.02, 0.1, 0.2, 0.5, 1.0]
    N_REPETICIONES = 50 # Número de veces que se repite la simulación para cada lambda
    WARMUP_MIN = 60

    resultados_finales = []
    historiales = []
    print("--- Iniciando Bucle Experimental ---")
    for lambda_val in lambdas_a_probar:
        print(f"\n--- Probando con λ = {lambda_val:.4f} ---")
        for i in range(N_REPETICIONES):
            resultado_run, df_hist = simular_una_corrida(lambda_val, warmup=WARMUP_MIN)
            resultado_run['lambda'] = lambda_val
            resultados_finales.append(resultado_run)
            df_hist['lambda'] = lambda_val
            df_hist['rep'] = i
            historiales.append(df_hist)
            print(".", end="")
    print("\n\n--- Bucle Experimental Finalizado ---")
    df_resultados = pd.DataFrame(resultados_finales)
    df_historiales = pd.concat(historiales, ignore_index=True)
    return df_resultados, df_historiales

if __name__ == '__main__':
    np.random.seed(42)
    random.seed(42)
    df_final_resultados, df_hist = ejecutar_experimentos()

In [ ]:
df_final_resultados['prop_congestion'] = (
    df_final_resultados['congestion_events'] / df_final_resultados['inbound_planes']
)
df_prop = df_final_resultados.groupby('lambda').agg(
    prop_promedio=('prop_congestion', 'mean'),
    prop_std=('prop_congestion', 'std'),
    n_simulaciones=('prop_congestion', 'count')
)

# Error estándar
df_prop['prop_se'] = df_prop['prop_std'] / np.sqrt(df_prop['n_simulaciones'])

df_estadisticas = df_final_resultados.groupby('lambda').agg(
    atraso_promedio=('total_delay_min', 'mean'),
    congestion_promedio=('congestion_events', 'mean'),
    desvios_promedio=('diversions', 'mean'),
    aterrizajes_promedio=('landed_planes', 'mean'),
    atraso_std=('total_delay_min', 'std'),
    congestion_std=('congestion_events', 'std'),
    desvios_std=('diversions', 'std'),
    n_simulaciones=('lambda', 'count'),
    total_aviones_promedio=('total_planes', 'mean'),
    total_aviones_std=('total_planes', 'std'),
    aviones_llegando_promedio=('inbound_planes', 'mean'),
    aviones_llegando_std=('inbound_planes', 'std'),
    aviones_activos_promedio=('active_planes', 'mean'),
    aviones_activos_std=('active_planes', 'std'),
    aviones_regresando_promedio=('returning_planes', 'mean'),
    aviones_regresando_std=('returning_planes', 'std'),
    cola_migraciones_promedio=('queue_size', 'mean'),
    cola_migraciones_std=('queue_size', 'std')
)

df_prop = df_prop.drop(columns=['prop_std', 'prop_promedio', 'n_simulaciones'])
df_estadisticas = df_estadisticas.merge(df_prop, left_index=True, right_index=True)

# 2. Calcular el Error Estándar de la Media (nuestro error de estimación)
df_estadisticas['atraso_error'] = 1.96 * df_estadisticas['atraso_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['congestion_error'] = 1.96 * df_estadisticas['congestion_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['desvios_error'] = 1.96 * df_estadisticas['desvios_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['total_aviones_error'] = 1.96 * df_estadisticas['total_aviones_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['aviones_llegando_error'] = 1.96 * df_estadisticas['aviones_llegando_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['cola_migraciones_error'] = 1.96 * df_estadisticas['cola_migraciones_std'] / np.sqrt(df_estadisticas['n_simulaciones'])

# 3. Calcular el atraso promedio POR AVIÓN
df_estadisticas['atraso_por_avion_promedio'] = df_estadisticas['atraso_promedio'] / df_estadisticas['aterrizajes_promedio']
df_estadisticas['atraso_por_avion_error'] = df_estadisticas['atraso_error'] / df_estadisticas['aterrizajes_promedio']

# 4. Congestion por minuto de simulación
df_estadisticas['congestion_por_minuto'] = df_estadisticas['congestion_promedio'] / df_estadisticas['aviones_llegando_promedio']
df_estadisticas['congestion_por_minuto_error'] = 1.96 * df_estadisticas['prop_se']
df_temp =  df_estadisticas['congestion_promedio'] / df_estadisticas['aviones_activos_promedio']

# 5. Desvíos normalizados sobre todos los aviones que aparecieron en promedio en total x simulación
df_final_filtrado = df_final_resultados.copy()
df_final_filtrado['prop_desvios'] = (
    df_final_filtrado['diversions'] / df_final_filtrado['total_planes']
)

df_prop_desvios = df_final_filtrado.groupby('lambda').agg(
    prop_promedio_desvios=('prop_desvios', 'mean'),
    prop_std=('prop_desvios', 'std'),
    n_simulaciones=('prop_desvios', 'count')
)

df_prop_desvios['prop_se_desvios'] = df_prop_desvios['prop_std'] / np.sqrt(df_prop_desvios['n_simulaciones'])
df_estadisticas = df_estadisticas.merge(df_prop_desvios[['prop_se_desvios', 'prop_promedio_desvios']], left_index=True, right_index=True)
df_estadisticas['prop_desvios_error'] = 1.96 * df_estadisticas['prop_se_desvios']

# 6. Aviones REGRESANDO por cada t sobre aviones en circulación
df_regresando = df_final_resultados.copy()
df_regresando['prop_regresos'] = (
    df_regresando['returning_planes'] / df_regresando['active_planes']
)

df_prop_regresando = df_regresando.groupby('lambda').agg(
    prop_promedio_regresos=('prop_regresos', 'mean'),
    prop_std=('prop_regresos', 'std'),
    n_simulaciones=('prop_regresos', 'count')
)

df_prop_regresando['prop_se_regresos'] = df_prop_regresando['prop_std'] / np.sqrt(df_prop_regresando['n_simulaciones'])
df_estadisticas = df_estadisticas.merge(df_prop_regresando[['prop_se_regresos', 'prop_promedio_regresos']], left_index=True, right_index=True)
df_estadisticas['regresando_error'] = 1.96 * df_estadisticas['prop_se_regresos']

# Eliminar atraso y stds
df_estadisticas = df_estadisticas.drop(columns=['atraso_promedio', 'atraso_std', 'atraso_error', 'congestion_std',
                                                 'desvios_std', 'congestion_promedio', 'congestion_error',
                                                 'desvios_promedio', 'desvios_error', 'aterrizajes_promedio',
                                                 'total_aviones_std', 'aviones_llegando_std', 'aviones_llegando_promedio',
                                                 'aviones_llegando_error', 'prop_se', 'prop_se_desvios',
                                                 'aviones_activos_promedio', 'aviones_activos_std',
                                                 'aviones_regresando_promedio', 'aviones_regresando_std',
                                                 'prop_se_regresos', 'cola_migraciones_std'])

# Mostramos la tabla final de resultados
print("--- Tabla de Estadísticas por Valor de λ ---")
display(df_estadisticas)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(
    df_estadisticas.index.astype(str),
    df_estadisticas['congestion_por_minuto'],
    yerr=df_estadisticas['congestion_por_minuto_error'],
    capsize=5,
    color='skyblue'
)
plt.title('Frecuencia de Congestiones vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Promedio de Eventos de Congestión por Día', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(
    df_estadisticas.index.astype(str),
    df_estadisticas['atraso_por_avion_promedio'],
    yerr=df_estadisticas['atraso_por_avion_error'],
    capsize=5,
    color='salmon'
)
plt.title('Atraso Promedio por Avión vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Atraso Promedio por Avión (Minutos)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(
    df_estadisticas.index.astype(str),
    df_estadisticas['prop_promedio_desvios'],
    yerr=df_estadisticas['prop_desvios_error'],
    capsize=5,
    color='lightcoral'
)
plt.title('Frecuencia de Desvíos vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Promedio de Desvíos por Día', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(
    df_estadisticas.index.astype(str),
    df_estadisticas['cola_migraciones_promedio'],
    yerr=df_estadisticas['cola_migraciones_error'],
    capsize=5,
    color='wheat'
)
plt.title('Tamaño de la cola de migraciones vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Cola promedio de migraciones por minuto', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Graficar barras apiladas de aviones activos y regresando
plt.figure(figsize=(10, 6))
bar_width = 0.4
indices = np.arange(len(df_estadisticas))
plt.bar(
    indices,
    1 - df_estadisticas['prop_promedio_regresos'] - df_temp,
    capsize=5,
    label='Aviones Activos',
    color='mediumseagreen',
    width=bar_width
)
plt.bar(
    indices,
    df_estadisticas['prop_promedio_regresos'],
    bottom=1 - df_estadisticas['prop_promedio_regresos'],
    capsize=5,
    label='Aviones Regresando',
    color='orange',
    width=bar_width
)
plt.bar(
    indices,
    df_temp,
    bottom=1 - df_estadisticas['prop_promedio_regresos'] - df_temp,
    capsize=5,
    label='Aviones Congestionados',
    color='none',
    edgecolor='gold',
    width=bar_width,
    hatch='//'
)
plt.xticks(indices, df_estadisticas.index.astype(str))
plt.title('Aviones Activos y Regresando vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Proporción de aviones', fontsize=12)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- Promediar sobre repeticiones ---
metrics = ["delay_avg"]

df_prom = (
    df_hist
    .groupby(["lambda", "minuto"], as_index=False)[metrics]
    .mean()
)

warmup = 60
for metric in metrics:
    plt.figure(figsize=(10,6))  # más ancho
    sns.lineplot(
        data=df_hist,              # usamos df_hist para que Seaborn calcule la variabilidad
        x="minuto",
        y=metric,
        hue="lambda",
        estimator="mean",          # promedio sobre repeticiones
        palette="deep"
    )
    plt.xlabel("Minuto de simulación")
    plt.ylabel(metric.replace("_", " ").capitalize())
    plt.title(f"Evolución de {metric.replace('_',' ')} en función del tiempo")
    plt.axvspan(0, warmup, color="grey", alpha=0.2)
    plt.grid(True)
    plt.show()


In [ ]:
 # --- Pasar de acumulado a métricas por minuto ---
df_hist_minuto = df_hist.copy().sort_values(by=["lambda", "rep", "minuto"])

for col in ["congestion_events", "diversions", "landed_planes"]:
    df_hist_minuto[col + "_per_min"] = (
        df_hist_minuto.groupby(["lambda", "rep"])[col]
        .diff()
        .fillna(0)
    )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")

warmup = 60  # minutos de warmup
window = 5   # ventana de smoothing en minutos

# --- Crear columnas suavizadas ---
df_smooth = df_hist_minuto.copy()
metrics = ["congestion_events_per_min", "diversions_per_min", "landed_planes_per_min"]

for lam in df_smooth['lambda'].unique():
    for rep in df_smooth['rep'].unique():
        mask = (df_smooth['lambda'] == lam) & (df_smooth['rep'] == rep)
        for col in metrics:
            df_smooth.loc[mask, col + "_smooth"] = df_smooth.loc[mask, col].rolling(
                window=window, min_periods=1
            ).mean()


# --- Función para graficar cada métrica ---
def plot_metric(metric, ylabel, title):
    plt.figure(figsize=(16, 6))
    sns.lineplot(
        data=df_smooth,
        x="minuto",
        y=metric + "_smooth",
        hue="lambda",
        estimator="mean",
        errorbar="sd",
        palette="deep"
    )
    # Zona de warmup
    plt.axvspan(0, warmup, color="grey", alpha=0.2)

    plt.xlabel("Minuto de simulación")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.show()

# --- Graficar ---
plot_metric("congestion_events_per_min", "Eventos de congestión", "Eventos de congestión por minuto (promedio)")
plot_metric("diversions_per_min", "Número de desvíos", "Desvíos por minuto (promedio)")
plot_metric("landed_planes_per_min", "Aterrizajes", "Aterrizajes por minuto (promedio)")